In [1]:
# import modules
import os,binascii
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from aesLongKeyGen16 import *

In [2]:
# Subroutine for encryption
def aesEncrypt(message_bytes, cipher):
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(message_bytes) + encryptor.finalize()
    return ciphertext

# Subroutine for decryption
def aesDecrypt(ciphertext, cipher):
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    return plaintext

# meet-in-the-middle attack to find 16 bit keys
def meetInTheMiddleAttack(plaintext, ciphertext):
    iv=b'\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0'
    plaintext = plaintext.encode('utf-8')
    ciphertext = binascii.unhexlify(ciphertext)

    encrypt1 = []
    for shortKey1 in range(0, 2**16):
        shortKey1 = bytearray(shortKey1.to_bytes(2, byteorder='big'))
        longKey1 = expandKey(shortKey1)
        cipher1 = Cipher(algorithms.AES(longKey1), modes.CBC(iv))
        ct = aesEncrypt(plaintext, cipher1)
        encrypt1.append([longKey1, ct, shortKey1])
    encrypt1.sort(key=lambda x: x[1])

    decrypt2 = []
    for shortKey2 in range(0, 2**16):
        shortKey2 = bytearray(shortKey2.to_bytes(2, byteorder='big'))
        longKey2 = expandKey(shortKey2)
        cipher2 = Cipher(algorithms.AES(longKey2), modes.CBC(iv))
        pt = aesDecrypt(ciphertext, cipher2)
        decrypt2.append([longKey2, pt, shortKey2])
    decrypt2.sort(key=lambda x: x[1], reverse=True)

    n = len(encrypt1)
    i,j = 0, n-1
    while i < n and j >= 0:
        if encrypt1[i][1] == decrypt2[j][1]:
            return (encrypt1[i][2], decrypt2[j][2])
        elif encrypt1[i][1] < decrypt2[j][1]:
            i += 1
        else:
            j -= 1
    return -1, -1

In [3]:
# read 2aesPlaintexts.txt and 2aesCiphertexts.txt
with open('2aesPlaintexts.txt', 'r') as f:
    plaintexts = f.read().splitlines()
with open('2aesCiphertexts.txt', 'r') as f:
    ciphertexts = f.read().splitlines()
print(f'Plaintexts: {plaintexts}')
print(f'Ciphertexts: {ciphertexts}')

Plaintexts: ['Hydrodynamometer', 'Circumnavigation', 'Crystallographer', 'Microphotography']
Ciphertexts: ['ea7f6a9b8ca5641e5c574000342a6322', '24194bf1995f73a675ddabddbde46c43', 'b7f2292330b32d43f351a9588bdfa640', '85c9b1e834c4c361db037023520fb438', 'c85afb6a2947ee3497ddf2b10e3ac81b']


In [4]:
# Tried to find key from 4th plaintext and ciphertext
key = meetInTheMiddleAttack(plaintexts[3], ciphertexts[3])
if key == (-1,-1):
    print("Key not found")
else:
    print(f'Key found: ')
    print(f'Key1: {key[0].hex()}')
    print(f'Key2: {key[1].hex()}')

Key found: 
Key1: b2df
Key2: 16c3


In [5]:
# find secret plaintext by decrypting secret ciphertext
# do the following if key is found
sCt = ciphertexts[-1]
sCt = binascii.unhexlify(sCt)
sKey1 = bytearray(key[0])
sKey2 = bytearray(key[1])
lKey1 = expandKey(sKey1)
lKey2 = expandKey(sKey2)
iv=b'\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0'
cipher1 = Cipher(algorithms.AES(lKey1), modes.CBC(iv))
cipher2 = Cipher(algorithms.AES(lKey2), modes.CBC(iv))
decryptor1 = cipher1.decryptor()
decryptor2 = cipher2.decryptor()
p2 = decryptor2.update(sCt) + decryptor2.finalize()
p1 = decryptor1.update(p2) + decryptor1.finalize()
print(f'Secret plaintext: {p1.decode("UTF-8")}')

# write keys, expanded keys, and secret plaintext to file
with open('secretInfo.txt', 'w') as f:
    f.write(f'Key1: {key[0].hex()}\n')
    f.write(f'Key2: {key[1].hex()}\n')
    f.write(f'Expanded Key1: {lKey1.hex()}\n')
    f.write(f'Expanded Key2: {lKey2.hex()}\n')
    f.write(f'Secret Plaintext: {p1.decode("UTF-8")}\n')

Secret plaintext: paddlingcanoeist


In [6]:
# find keys for all plaintexts and ciphertexts
keys = []
for i in range(0, len(plaintexts)):
    shortKey = meetInTheMiddleAttack(plaintexts[i], ciphertexts[i])
    keys.append([shortKey[0].hex(), shortKey[1].hex()])
    if shortKey == (-1,-1):
        print(f'Key not found for plaintext {i+1}')
    else:
        print(f'for plaintext: {plaintexts[i]} and ciphertext: {ciphertexts[i]}')
        print(f'Key1: {shortKey[0].hex()}')
        print(f'Key2: {shortKey[1].hex()}')

print(f'Keys: {keys}')


for plaintext: Hydrodynamometer and ciphertext: ea7f6a9b8ca5641e5c574000342a6322
Key1: b2df
Key2: 16c3
for plaintext: Circumnavigation and ciphertext: 24194bf1995f73a675ddabddbde46c43
Key1: b2df
Key2: 16c3
for plaintext: Crystallographer and ciphertext: b7f2292330b32d43f351a9588bdfa640
Key1: b2df
Key2: 16c3
for plaintext: Microphotography and ciphertext: 85c9b1e834c4c361db037023520fb438
Key1: b2df
Key2: 16c3
Keys: [['b2df', '16c3'], ['b2df', '16c3'], ['b2df', '16c3'], ['b2df', '16c3']]
